In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import xgboost as xgb
#https://habrahabr.ru/company/ods/blog/327242/

import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
%matplotlib inline
import seaborn as sb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import *

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import __version__
from plotly import graph_objs as go

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

C:\Users\TEMP\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Читаем данные
df=pd.read_pickle('MG_Sales.pickle',compression='gzip')
celebrate=pd.read_pickle('celebrate.pickle')

In [203]:
#готовим выборку обучения
begin_period=dt.datetime(2015,1,1)
end_period=dt.datetime(2017,3,31)


#last=time_series.iloc[-1].name
#startDt=dt.datetime(last.year,last.month,last.day)
#lastDay=dt.datetime(last.year,last.month,1)-dt.timedelta(seconds=1)
#startmonth=dt.datetime(lastDay.year,lastDay.month,1)

start_forecast=end_period+dt.timedelta(days=1)
end_forecast=start_forecast+dt.timedelta(days=30)

query="Дата>=20150101 and ТоварЦеноваяГруппа=='12000-15000'"
time_series=pd.DataFrame(data=df.query(query).groupby('Дата')['Количество'].sum())
time_series.index.name='ds'
time_series.columns=['y']
time_series.y=time_series.y

#Пустой период в днях обучающей выборки
delta=end_period-begin_period
dummy_train = pd.DataFrame(index=np.array([begin_period + dt.timedelta(days=x) for x in range(0, delta.days)]).astype('datetime64[D]'))
dummy_train.index.name='ds'

#сливаем обучающую выборку и пустой период чтобы избежать пропусков дат, пропуски заполняем нулями
time_series=dummy_train.merge(time_series,left_index=True, right_index=True,how='outer').fillna(0)  

In [319]:
month_lags=1

def setPercentil(row,llim,ulim,med):
    if row<llim: return -2
    if row>ulim: return 2
    if row<med[0]: return -1
    if row>med[1]: return 1
    return 0
    

#временные характеристики
def setNewValues(_ts,cel):
    ts=_ts.copy()
    y=None
    week_d=None
    last_tempo=None
    
    ts['День недели'] = ts.index.weekday
    ts['Выходной'] = ts.index.weekday.isin([5,6])*2+ts.index.weekday.isin([4])*1
    #ts['Пятница'] = ts.index.weekday.isin([4])*1
    ts['Неделя'] = ts.index.week
    ts['Год'] = ts.index.year
    ts['Месяц'] = ts.index.month
    ts['День месяца'] = ts.index.day
    ts['День года'] = ts.index.dayofyear
    ts['Праздник']=0#Обычный день
    ts.loc[ts.index.isin(cel['Праздник']),'Праздник']=1#праздник
    
    if 'y' in ts.columns:#ТОЛЬКО для обучающей выборки
        #Фомируем характеристики модели    
        #Фиксируем аномально низкие и высокие продажи
        ul=5#Персентиль высоких продаж 2
        ll=5#7#Персентиль низких продаж 10
        md=25#ширина медианы

        ulim=np.percentile(ts['y'], 100.-ul)
        llim=np.percentile(ts['y'], ll)
        med=np.percentile(ts['y'], [50-md,50+md])
        
        #print(llim)

        #ts['Квантили']=0
        
        mean_dict_day=dict(ts.groupby(['День года'])['y'].mean())
        print(mean_dict_day)
        ts['Квантили']=ts.apply(lambda row: setPercentil(mean_dict_day[row['День года']],llim,ulim,med), axis=1)
        
        #ts.loc[ts.y<med[0],'Квантили']=-1
        #ts.loc[ts.y>med[1],'Квантили']=1
        #ts.loc[ts.y<llim,'Квантили']=-2
        #ts.loc[ts.y>ulim,'Квантили']=2
            
        #порядок дней в сезонности недельной продажи за исключением аномалий
        week_d=pd.DataFrame(data=ts[ts['Квантили']==0].groupby('День недели')['y'].sum().sort_values())
        week_d.insert(0,'Недельная сезонность',list(range(week_d.shape[0])))
        for i in list(set(range(7))-set(week_d.index.values)):
            week_d.loc[i,'Недельная сезонность']=-1
        ts=weekseason(ts,week_d)
    
        #Среднее и тренды
        #mean_dict=dict(ts.groupby(['Год','Неделя'])['y'].median())
        #ts['Среднее по неделе']=ts.apply(lambda row: mean_dict[row['Год'],row['Неделя']] , axis=1)        
        median_dict=dict(ts.groupby(['Год'])['y'].median())
        #ts['Медиана за год']=ts.apply(lambda row: median_dict[row['Год']] , axis=1)
        mean_dict=dict(ts.groupby(['Год'])['y'].mean())
        ts['Среднее за год']=ts.apply(lambda row: mean_dict[row['Год']] , axis=1)
        #mean_dict=dict(ts.groupby(['День года'])['y'].mean())
        ts['Среднее по дню года']=ts.apply(lambda row: mean_dict_day[row['День года']] , axis=1)        
                
        #mean_month=np.log(ts.groupby(['Год','Месяц'])['y'].median())#)dict(
        #ts['Медиана по месяцу']=ts.apply(lambda row: mean_dict[row['Год'],row['Месяц']] , axis=1)  
        #for i in range(1,month_lags+1):
        #    shift_month=dict(mean_month.shift(i).fillna(0))
        #    ts['Прошлый месяц {}'.format(i)]=ts.apply(lambda row: shift_month[row['Год'],row['Месяц']] , axis=1)
        #last_month_median=mean_month.tail(-1).values[0]
        
        #РАССЧИТЫВАЕМ коэффициент роста
        mean_month=ts.groupby(['Год','Месяц'])['y'].mean()
        mean_month_shift=mean_month.shift(1).fillna(0)
        #(mean_month/mean_month_shift-1)*mean_month.shift(11).fillna(0)
        inc_temp=(mean_month/mean_month_shift-1)*mean_month.shift(11).fillna(0)
        last_tempo=inc_temp.tail(1).values[0]
        #mean_dict=dict(inc_temp.shift(1).fillna(1))
        #ts['Темп прироста']=ts.apply(lambda row: mean_dict[row['Год'],row['Месяц']], axis=1) 
        
        #ts['Недельный тренд']=ts['Среднее по неделе'].diff(7).fillna(0)
    
        #Вычленяем целевую переменную
        y=ts.y
        ts.drop(['y'], axis=1, inplace=True)
        
        #заглушки
        #ts.drop(['Месяц'], axis=1, inplace=True)
        #ts['Недельный тренд']=0
        #ts['Медиана по месяцу']=0
        ts.drop(['День недели'], axis=1, inplace=True)
        #ts.drop(['Медиана по месяцу'], axis=1, inplace=True)
        #ts['Среднее по дню года']=0
        #ts['Медиана за год']=0
        #ts['Среднее за год']=0
        #ts['Среднее по неделе']=0  
        
        #ts['Темп прироста']=0
        #last_tempo=0
        
        #last_month_median=0
        #for i in range(1,month_lags+1):            
        #    ts['Прошлый месяц {}'.format(i)]=0
        
    
    return ts,y,week_d,last_tempo


def weekseason(ts,week_d):
    ts['Недельная сезонность']=ts['День недели'].map(lambda cell: week_d.loc[cell,'Недельная сезонность'])
    return ts

#Сдвигаем период на год вперед
def fillTimeSeriesForecast(_tsf,ts,week_d,last_tempo):
    cols=['Квантили','Среднее по дню года']#'Недельный тренд',,'Среднее по неделе'
        
    tsf=weekseason(_tsf.copy(),week_d)
    tsf.drop(['День недели'], axis=1, inplace=True)
               
    #перебрасываем данные за прошлый год
    year=tsf.iloc[0].name.year
    tempts=ts.loc[dt.datetime(year-1,1,1):dt.datetime(year-1,12,31),cols].shift(1,DateOffset(years=1))
    
    #print(ts.loc[dt.datetime(2016,4,1):dt.datetime(2016,4,10)])
            
    #проверяем 29 февраля    
    feb28=dt.datetime(year,2,28)
    feb29_ts=tempts.loc[feb28]
    if tempts.shape[0]==366:#Прошлый год был високосным                
        tempts.reset_index(inplace=True)#снимаем индекс чтобы удалить строчки
        tempts.drop(tempts.loc[tempts.ds==feb28].index, inplace=True)#удаляем строчки за два дня
        tempts.set_index('ds',inplace=True)#восстанавливаем индекс
        
        if (tsf.iloc[0].name<=feb28) & (tsf.iloc[-1].name>=feb28):#ЕСЛИ попадаем в период
            #берем среднее за два дня 28 и 29
            tempts.loc[feb28]=feb29_ts.mean()
        
        tsf[cols]=tempts
    else:
        try:
            feb29=dt.datetime(year,2,29)#этот год високосный            
            if (tsf.iloc[0].name<=feb29) & (tsf.iloc[-1].name>=feb29):#ЕСЛИ попадаем в период
                #берем среднее за 28 февраля и 1 марта
                tempts.loc[feb29]=tempts.loc[feb28:dt.datetime(year,3,1)].mean()#берем среднее за два дня 28 и 29                
            
            tsf[cols]=tempts            
        except:            
            tsf[cols]=tempts
    
    
    #Мы знаем средние данные за год, учитываем как тренд
    last_vals=ts.tail(1)
    for col in ['Среднее за год']:#,'Медиана за год'
        tsf[col]=last_vals[col].values[0]
        
    print('Темп прироста: ',last_tempo)
    #tsf['Среднее по дню года']+=last_tempo#<---------------------------------
    #tsf['Темп прироста']=0#last_tempo
    
    
    #ДАННЫЕ за прошлые месяца    
    #for i in range(2,month_lags+1):        
     #   tsf['Прошлый месяц {}'.format(i)]=ts.tail(1)['Прошлый месяц {}'.format(i-1)].values[0]
    #данные за последний месяц    
    #tsf['Прошлый месяц 1']=last_month_median
    
    #tsf['Среднее по дню года']=tsf['Среднее по дню года']*0.775280899

    #tsf['Медиана за год']=tsf['Медиана за год']*.3
    
    
    
        
    return tsf[ts.columns]#Колонки в правильном порядке
    

def createTimeSeriesForecast(begin, end,cel):
    date_list = pd.date_range(begin, end).tolist()
    time_series_forecast=pd.DataFrame(index=date_list)
    time_series_forecast.index.name='ds'    
    
    time_series_forecast=setNewValues(time_series_forecast,cel)
    
    return time_series_forecast    

In [320]:
lastDay=end_period
#last=time_series.iloc[-1].name
#startDt=dt.datetime(last.year,last.month,last.day)
#lastDay=dt.datetime(last.year,last.month,1)-dt.timedelta(seconds=1)
#startmonth=dt.datetime(lastDay.year,lastDay.month,1)

time_series_train, y_train,week_d,last_tempo=setNewValues(time_series.loc[:lastDay],celebrate)
    
tss = TimeSeriesSplit(n_splits=10)
tss_cv=list(tss.split(time_series_train,y_train))

# задаём параметры
params = {
        'objective': 'reg:linear',
        'booster':'gblinear',                
        'eta': 0.27,#коэффициент обучения
        'alpha': 0.0,
        'lambda_bias': 0,
        'eval_metric': 'rmse'
    }

#Нормализуйте обучающую выборку с помощью класса StandardScaler
scaler = StandardScaler(with_mean=True,with_std=True)
dtrain = xgb.DMatrix(time_series_train, label=y_train)#scaler.fit_transform(

# прогоняем на кросс-валидации с метрикой rmse
cv = xgb.cv(params, dtrain, metrics = ('rmse'), early_stopping_rounds=50,verbose_eval=False, show_stdv=False, num_boost_round=1500,folds=tss_cv)#,nfold=10

mod_n=cv['test-rmse-mean'].argmin()

deviation = cv.loc[mod_n]["test-rmse-mean"]
print('Средняя ошибка на кросс-валидации: ',deviation,', номер модели: ',mod_n)
best_iteration=mod_n

#Модель предсказания
time_series_forecast,_,__,____ =createTimeSeriesForecast(start_forecast, end_forecast,celebrate)
time_series_forecast=fillTimeSeriesForecast(time_series_forecast,time_series_train,week_d,last_tempo)

{1: 7.333333333333333, 2: 43.0, 3: 42.333333333333336, 4: 41.333333333333336, 5: 39.0, 6: 33.666666666666664, 7: 33.666666666666664, 8: 33.666666666666664, 9: 24.666666666666668, 10: 27.333333333333332, 11: 25.333333333333332, 12: 31.333333333333332, 13: 44.666666666666664, 14: 35.333333333333336, 15: 29.0, 16: 41.333333333333336, 17: 46.0, 18: 34.333333333333336, 19: 23.333333333333332, 20: 25.0, 21: 35.666666666666664, 22: 41.666666666666664, 23: 36.0, 24: 39.666666666666664, 25: 38.333333333333336, 26: 41.333333333333336, 27: 42.666666666666664, 28: 38.333333333333336, 29: 29.333333333333332, 30: 40.0, 31: 41.333333333333336, 32: 37.0, 33: 26.333333333333332, 34: 34.0, 35: 38.666666666666664, 36: 41.333333333333336, 37: 25.0, 38: 34.0, 39: 31.333333333333332, 40: 30.666666666666668, 41: 28.666666666666668, 42: 45.333333333333336, 43: 39.666666666666664, 44: 54.0, 45: 53.0, 46: 24.0, 47: 26.333333333333332, 48: 26.0, 49: 32.0, 50: 28.333333333333332, 51: 24.666666666666668, 52: 38.33

In [301]:
bst = xgb.train(params, dtrain, best_iteration,verbose_eval=False)
prediction_test = pd.DataFrame(data=bst.predict(xgb.DMatrix(time_series_forecast)),index=time_series_forecast.index)#scaler.transform(
prediction_test[0]=prediction_test[0].map(lambda val: 0 if val<0.01 else round(val,3))

In [302]:
y_test=time_series[start_forecast:end_forecast]
#print ('MAE =',mean_absolute_error(y_test.y,prediction_test[0]))
print('RSME = ',np.sqrt(mean_squared_error(y_test.y,prediction_test[0])))

init_notebook_mode(connected = True)
trace1 = go.Scatter(
            x = prediction_test.index,
            y = prediction_test[0],
            mode = 'lines',            
            name = 'Прогноз 2017',
            line=dict(
                shape='spline',
                dash = 'dash',
                width = 3
            )
        )       

ty=(time_series[dt.datetime(2016,1,1):dt.datetime(2016,6,1)].shift(1,DateOffset(years=1)))
trace0 = go.Scatter(
            x = ty.index,
            y = ty.y,
            mode = 'lines',            
            name = '2016 год',
            line=dict(
                shape='spline',
                dash = 'dot'
            )
        )       

ty=(time_series[dt.datetime(2015,1,1):dt.datetime(2015,6,1)].shift(2,DateOffset(years=1)))
trace5 = go.Scatter(
            x = ty.index,
            y = ty.y,
            mode = 'lines',            
            name = '2015 год',
            line=dict(
                shape='spline',
                dash = 'dot'
            )
        )   

y_test=time_series.loc[time_series_forecast.index]
trace2 = go.Scatter(
            x = y_test.index,
            y = y_test.y,
            mode = 'lines',            
            name = 'Продажи 2017',
            line=dict(
                shape='spline'
            )
        ) 
trace4 = go.Scatter(
            x = y_test.index,
            y = y_test.y-prediction_test[0],
            mode = 'markers',
            name = 'Разница прогноза и данных',
            line=dict(
                shape='spline',
                dash = 'dot'
            )
        ) 

layout = go.Layout(
    title=query
)
    
fig =go.Figure(data = [trace1,trace2,trace5,trace0,trace4],layout=layout)#,trace0
iplot(fig, show_link=False)



RSME =  9.17957747288


#Кросс-валидация по времени
last=time_series.iloc[-1].name
startDt=dt.datetime(last.year,last.month,last.day)
#lastDay=dt.datetime(last.year,last.month,1)-dt.timedelta(seconds=1)
startmonth=dt.datetime(startDt.year,startDt.month,1)

pediods=[]
for i in range(12):
    pediods.append([startmonth,startDt])    
    startDt=startmonth-dt.timedelta(seconds=1)
    startmonth=dt.datetime(startDt.year,startDt.month,1)
    
rmse=[]
best_iteration=[]
for begin,end in reversed(pediods):
    date_div_past=begin-dt.timedelta(days=1)
            
    #ГОТОВИМ модель обучения
    time_series_train, y_train,week_d=setNewValues(time_series.loc[:date_div_past],celebrate)
    
    #Модель предсказания
    time_series_forecast,_,__ =createTimeSeriesForecast(begin, end,celebrate)
    time_series_forecast=fillTimeSeriesForecast(time_series_forecast,time_series_train,week_d)
    
        
    #Нормализуйте обучающую выборку с помощью класса StandardScaler
    scaler = StandardScaler(with_mean=True,with_std=True)
    dtrain = xgb.DMatrix(scaler.fit_transform(time_series_train), label=y_train)
    
    # задаём параметры
    params = {
        'objective': 'reg:linear',
        'booster':'gblinear',
        'tree_method': 'exact',                
        'eta': 0.17,#коэффициент обучения
        'alpha': 0.01,
        'lambda_bias': 0,
        'eval_metric': 'rmse'
    }
   
    #Обучаем
    validate = xgb.DMatrix(scaler.transform(time_series_forecast), label=time_series.loc[time_series_forecast.index])
    watchlist  = [(dtrain,'train'), (validate, 'eval')]
    
    bst = xgb.train(params, dtrain, 1000,watchlist,early_stopping_rounds=25,verbose_eval=False)# evals_result=progress progress = dict()
        
    best_iteration.append(bst.best_iteration)
    rmse.append(bst.best_score)
    
mean_rmse=np.mean(rmse)
best_iteration=int(np.ceil(np.median(best_iteration)))
print('rmse=',mean_rmse)

print('best_iteration=',best_iteration)

In [ ]:
time_series['Год'] = time_series.index.year
time_series['Месяц'] = time_series.index.month
mean_month=time_series.groupby(['Год','Месяц'])['y'].median()
median_dict=time_series.groupby(['Год'])['y'].median()

mean_month_shift=mean_month.shift(1).fillna(1)
#(mean_month/mean_month_shift-1)*mean_month.shift(11).fillna(0)

In [306]:
time_series_forecast.tail(10)

,Выходной,Неделя,Год,Месяц,День месяца,День года,Праздник,Квантили,Недельная сезонность,Среднее за год,Среднее по дню года
ds,,,,,,,,,,,
2017-04-22,2,16,2017,4,22,112,0,0,6,28.722222,23.0
2017-04-23,2,16,2017,4,23,113,0,0,2,28.722222,22.5
2017-04-24,0,17,2017,4,24,114,0,1,0,28.722222,42.0
2017-04-25,0,17,2017,4,25,115,0,-2,1,28.722222,10.0
2017-04-26,0,17,2017,4,26,116,0,0,3,28.722222,24.5
2017-04-27,0,17,2017,4,27,117,0,-1,5,28.722222,17.0
2017-04-28,1,17,2017,4,28,118,0,-1,4,28.722222,18.5
2017-04-29,2,17,2017,4,29,119,0,0,6,28.722222,23.5
2017-04-30,2,17,2017,4,30,120,0,-1,2,28.722222,15.0


In [316]:
time_series.loc[dt.datetime(2016,4,20):dt.datetime(2016,4,30)]

,y
ds,
2016-04-20,18
2016-04-21,33
2016-04-22,27
2016-04-23,25
2016-04-24,42
2016-04-25,5
2016-04-26,24
2016-04-27,10
2016-04-28,15


In [269]:
print(time_series.loc[dt.datetime(2016,4,1):dt.datetime(2016,4,10)])

             y
ds            
2016-04-01  25
2016-04-02  23
2016-04-03  23
2016-04-04  17
2016-04-05  12
2016-04-06  23
2016-04-07  14
2016-04-08  17
2016-04-09  33
2016-04-10  25
